In [1]:
import argparse
import random

import numpy as np
import torch

import config
import dataset
from informer_model import InformerModel
from model import Model
from moment_model import MomentModel
from propose import ProposedModel

In [2]:
def set_seed(seed: int) -> None:
    # random
    random.seed(seed)

    # numpy
    np.random.seed(seed)

    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.mps.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def load_moment_model(args: argparse.Namespace) -> Model:
    return MomentModel(param="AutonLab/MOMENT-1-large", pred_len=args.pred_len)


def load_informer_model(args: argparse.Namespace) -> Model:
    return InformerModel(args, checkpoint_path="checkpoints/informer.pth")


def load_proposed_model(moment_model: Model, informer_model: Model) -> Model:
    return ProposedModel(moment_model=moment_model, informer_model=informer_model)

In [3]:
set_seed(0)

args = config.ARGS
_, test_dataset = dataset.load_dataset(args=args)

moment_model = load_moment_model(args=args)
informer_model = load_informer_model(args=args)
proposed_model = load_proposed_model(moment_model, informer_model)

config.json: 100%|██████████| 951/951 [00:00<?, ?B/s] 
pytorch_model.bin: 100%|██████████| 1.39G/1.39G [02:00<00:00, 11.5MB/s]


In [ ]:
# 予測結果の評価
for method, model in {
    "informer": informer_model,
    "moment": moment_model,
    "proposed": proposed_model,
}.items():
    print(f"testing: {method}")
    test_dataloader = dataset.to_dataloader(test_dataset, args, "test")

    for batch in test_dataloader:
        _, batch_y, _, _ = batch
        y_true = batch_y[:, -1].squeeze()

        # TODO: モデルの予測結果の出力と評価をする

testing: informer
tensor([[-0.9993,  0.0131],
        [-0.8792,  0.0356],
        [-0.8391,  0.0131],
        [-0.9249,  0.0574],
        [-0.9220,  0.0342],
        [-0.8531,  0.0166],
        [-0.9568,  0.0733],
        [-0.9650,  0.0272],
        [-0.8782,  0.0141],
        [-0.9174,  0.0833],
        [-0.9460,  0.0301],
        [-0.8469,  0.0223],
        [-0.8728,  0.0677],
        [-0.9321,  0.0235],
        [-0.7540,  0.0330],
        [-0.6869,  0.0164],
        [-0.8550,  0.0182],
        [-0.7874,  0.0317],
        [-0.7121,  0.0163],
        [-0.8474,  0.0353],
        [-0.7808,  0.0400],
        [-0.7276,  0.0130],
        [-0.8566,  0.0563],
        [-0.8759,  0.0350],
        [-0.8395,  0.0148],
        [-0.9142,  0.0700],
        [-1.0128,  0.0339],
        [-0.9012,  0.0585],
        [-0.9868,  0.0539],
        [-1.0785,  0.0096],
        [-1.0187,  0.0201],
        [-1.0386,  0.0123]], grad_fn=<StackBackward0>) tensor([-0.8546, -0.8700, -0.9543, -0.9313, -0.9849, -1.023